In [1]:
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy
from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import ast
import nltk
# For See all Columns and Rows 
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

KeyboardInterrupt: 

In [ ]:
credits = pd.read_csv('credits.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
# Merge Movies and Credit_df Dataset
movies = movies.merge(credits,on='title')

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew','popularity','spoken_languages','vote_count']]
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
#Now doing text vectorization using the Bag of Words technique
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    # Check if the movie exists in the DataFrame
    if movie not in new_df['title'].values:
        print(f"Movie '{movie}' not found in the dataset.")
        return
    
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        oof = new_df.iloc[i[0]]
        print(new_df.iloc[i[0]].title)